In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# download dataset
!kaggle datasets download -d guiferviz/rsna_stage1_png_128
# unzip dataset
!unzip rsna_stage1_png_128.zip

Streaming output truncated to the last 5000 lines.
  inflating: stage_1_train_images/ID_fe1915645.png  
  inflating: stage_1_train_images/ID_fe19193b3.png  
  inflating: stage_1_train_images/ID_fe19266e1.png  
  inflating: stage_1_train_images/ID_fe192f4f6.png  
  inflating: stage_1_train_images/ID_fe193f9de.png  
  inflating: stage_1_train_images/ID_fe196108f.png  
  inflating: stage_1_train_images/ID_fe19784ac.png  
  inflating: stage_1_train_images/ID_fe19a2c4f.png  
  inflating: stage_1_train_images/ID_fe19a4e64.png  
  inflating: stage_1_train_images/ID_fe19b70a2.png  
  inflating: stage_1_train_images/ID_fe19e5443.png  
  inflating: stage_1_train_images/ID_fe19f44ba.png  
  inflating: stage_1_train_images/ID_fe1a04a3d.png  
  inflating: stage_1_train_images/ID_fe1a2a02a.png  
  inflating: stage_1_train_images/ID_fe1a3a548.png  
  inflating: stage_1_train_images/ID_fe1a50ea5.png  
  inflating: stage_1_train_images/ID_fe1a8305d.png  
  inflating: stage_1_train_images/ID_fe1ab85eb.p

In [ ]:
csv_path = '/content/stage_1_train.csv'
df = pd.read_csv(csv_path, header=None, names=['ID_Label', 'Label'])

<ipython-input-21-2f91f1d967bb>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, header=None, names=['ID_Label', 'Label'])


In [ ]:
df['image_id'] = df['ID_Label'].str.split('_').str[:2].str.join('_')

df_any = df[df['ID_Label'].str.endswith('_any')].copy()

In [ ]:

image_folder = '/content/stage_1_train_images'
df_any['image_path'] = df_any['image_id'].apply(lambda x: os.path.join(image_folder, x + '.png'))

In [ ]:
df_any.head()

,ID_Label,Label,image_id,image_path
6,ID_63eb1e259_any,0,ID_63eb1e259,/content/stage_1_train_images/ID_63eb1e259.png
12,ID_2669954a7_any,0,ID_2669954a7,/content/stage_1_train_images/ID_2669954a7.png
18,ID_52c9913b1_any,0,ID_52c9913b1,/content/stage_1_train_images/ID_52c9913b1.png
24,ID_4e6ff6126_any,0,ID_4e6ff6126,/content/stage_1_train_images/ID_4e6ff6126.png
30,ID_7858edd88_any,0,ID_7858edd88,/content/stage_1_train_images/ID_7858edd88.png


In [ ]:
print(df_any["image_path"].values)

['/content/stage_1_train_images/ID_63eb1e259.png'
 '/content/stage_1_train_images/ID_2669954a7.png'
 '/content/stage_1_train_images/ID_52c9913b1.png' ...
 '/content/stage_1_train_images/ID_a9797cb3a.png'
 '/content/stage_1_train_images/ID_9375f67bd.png'
 '/content/stage_1_train_images/ID_5c8b5d701.png']


In [ ]:
df_any['Label'] = df_any['Label'].astype(str)

In [ ]:
# Separate the data into two groups: one for Label == '1' and one for Label == '0'
df_class_1 = df_any[df_any['Label'] == '1']
df_class_0 = df_any[df_any['Label'] == '0']

In [ ]:
# Check the sizes to ensure you have enough data
print(f"Class 1 samples: {len(df_class_1)}")
print(f"Class 0 samples: {len(df_class_0)}")

Class 1 samples: 97103
Class 0 samples: 577159


In [ ]:
# Randomly sample 25,000 from each class
df_class_1_sample = df_class_1.sample(n=25000, random_state=42)
df_class_0_sample = df_class_0.sample(n=25000, random_state=42)

In [ ]:
# Combine the two samples
df_balanced = pd.concat([df_class_1_sample, df_class_0_sample])

In [ ]:
# Shuffle the combined dataframe to mix class 1 and class 0 samples
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Check the resulting class distribution
label_counts_balanced = df_balanced['Label'].value_counts()
print(label_counts_balanced)

Label
0    25000
1    25000
Name: count, dtype: int64


In [ ]:
# The first 50000 balanced records
df_first_50000_balanced = df_balanced.head(50000)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(df_first_50000_balanced, test_size=0.1, random_state=42)

In [ ]:
# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 45000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


# MODEL BUILDING

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.models import Model

In [ ]:
xception = Xception(include_top = False,input_shape=(224,224,3))

In [ ]:
xception.summary()
for layer in xception.layers:
  layer.trainable = False

Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 20,806,952 (79.37 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [ ]:
x = Flatten()(xception.output)

In [ ]:
output = Dense(1,activation = 'sigmoid')(x)

In [ ]:
class_model= Model(xception.input,output)

In [ ]:
class_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├──────────────────────

 Total params: 20,961,833 (79.96 MB)

 Trainable params: 100,353 (392.00 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [ ]:
class_model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [ ]:
r = class_model.fit(train_generator,validation_data = val_generator,epochs=10 )

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 787s 545ms/step - accuracy: 0.7067 - loss: 1.1303 - val_accuracy: 0.7636 - val_loss: 0.8621
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 786s 544ms/step - accuracy: 0.7360 - loss: 1.1520 - val_accuracy: 0.7440 - val_loss: 0.8982
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 841s 571ms/step - accuracy: 0.7389 - loss: 1.1776 - val_accuracy: 0.7906 - val_loss: 0.8541
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 960s 678ms/step - accuracy: 0.7402 - loss: 1.2692 - val_accuracy: 0.7488 - val_loss: 1.0862
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 943s 650ms/step - accuracy: 0.7505 - loss: 1.1586 - val_accuracy: 0.7698 - val_loss: 1.1512
Epoch 6/10
 650/1407 ━━━━━━━━━━━━━━━━━━━━ 8:01 636ms/step - accuracy: 0.7553 - loss: 1.1344

In [ ]:
import matplotlib.pyplot as plt
# plot the accuracy
plt.title("Accuracy Curve of Xception model")
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
plt.title("Loss Curve of Xception model")
# plot the loss
plt.title("Loss Curve of VGG 16  model")
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')